This is a page for listing the peat health indicators for a given site

In [ ]:
import panel as pn
import param

import utils.cards
import utils.location
import utils.peat_health_indicators.models
import utils.peat_health_indicators.phi
import utils.template
import utils.utils

pn.extension(loading_indicator=True, defer_load=True)

In [ ]:
def get_card_for_phi(directory: str) -> pn.Card:
    """Get a card for the given site level peat health indicator (PHI)."""

    phi = utils.peat_health_indicators.phi.DailyPHI.from_directory(directory)

    daily_phi = utils.utils.get_url(
        "daily-phi",
        {"site-id": phi.site_id, "indicator-id": phi.name},
    )
    annual_phi = utils.utils.get_url(
        "annual-phi",
        {"site-id": phi.site_id, "indicator-id": phi.name},
    )

    return pn.Card(
        objects=[
            pn.pane.HoloViews(
                phi.map(),
                width=600,
                height=500,
            ),
            pn.pane.Markdown(phi.description),
            pn.pane.Markdown(
                f"""
                🔗 [Explore daily peat health indicators]({daily_phi}) or
                🔗 [Explore annual peat health indicators]({annual_phi})
                """
            ),
        ],
        collapsed=True,
        title=phi.name,
        sizing_mode="stretch_width",
    )

In [ ]:
class Page(utils.location.UrlQueryParams):
    @param.depends("site", watch=False)
    def get_phi_cards(self) -> list:
        if self.site is None:
            return ["No indicators found"]

        # mapping from site-id -> extent name -> subdirectory
        sites = utils.peat_health_indicators.models.all_site_level_peat_health_indicators()

        # mapping from extent name -> subdirectory
        extents = sites.get(self.site.id, {})

        return [get_card_for_phi(directory) for directory in extents.values()]

    @param.depends("site", watch=False)
    def main(self) -> pn.Column:
        """content to render in the main area of a template"""
        elements = []

        if self.site is None:
            elements.append(pn.pane.Alert("Site not found", alert_type="danger"))
        else:
            site_name = self.site.title or self.site.id
            elements.append(
                pn.pane.Markdown(
                    f"""
                    # Peat health indicators

                    On this page you can discover the peat health indicators for the site {site_name}.

                    Each of the data products below is based on a specific peat extent map or area of interest,
                    over which site-level peat health indicators are generated,
                    at both daily and annual time steps.

                    Check out one of the data products below to explore the peat health indicators.
                    """
                )
            )
            elements.append(pn.pane.Markdown("## Site"))
            elements.append(utils.cards.site(self.site, collapsed=True))
            elements.append(pn.pane.Markdown("## Data products"))
            cards = self.get_phi_cards()  # type: ignore
            if len(cards) == 0:
                elements.append(pn.pane.Alert("No peat health indicators available", alert_type="info"))
            else:
                elements.extend(cards)

        return pn.Column(
            *elements,
            align=("start", "center"),
            min_width=400,
            max_width=1200,
            sizing_mode="stretch_width",
        )


In [ ]:
page = Page()

In [ ]:
# In a notebook environment pn.state.location is not initialized until the first plot has been displayed

pn.state.location.sync(page, {"site_id": "site-id"})  # type: ignore

In [ ]:
template = utils.template.get_template(main=page.main)
template.servable();